In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


* loader

In [11]:
import os
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pprint import pprint

# PDF 파일 경로 설정
pdf_filepath = '../data/콘텐츠분쟁해결_사례.pdf'

# 파일 존재 여부 확인 (파일이 없으면 오류 발생)
if not os.path.exists(pdf_filepath):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {pdf_filepath}")

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader('../data/콘텐츠분쟁해결_사례.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")


==> 1. 문서 로딩 → PDF 읽기...
  총 109페이지 로드 완료


In [15]:

print("==> 2. 문서 분할 → 작은 청크로 나누기")
pythontext_splitter  = RecursiveCharacterTextSplitter(
    chunk_size=1500,        #  법률 사례는 1200-1800자 권장
    chunk_overlap=300,      #  사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n", "\n\n", "\n", ".", " ", ""
    ]
)


chunks = pythontext_splitter .split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")


==> 2. 문서 분할 → 작은 청크로 나누기
  104개 청크 생성 완료
  평균 청크 길이: 722자


In [16]:

print("==> 3. 벡터화 → 임베딩으로 변환")
pythonembeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  #  한국어 법률 용어에 최적
    dimensions=1536                  #  고성능 임베딩 차원
)


==> 3. 벡터화 → 임베딩으로 변환


In [18]:

print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, pythonembeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")



==> 4. 저장 → FAISS 벡터스토어에 저장
 FAISS 벡터스토어 생성 완료 (104개 벡터)


In [19]:

print("===> 5. 검색 → 질문과 유사한 문서 찾기")
pythonretriever = vectorstore.as_retriever(
    search_type="similarity",        #  또는 "mmr" (다양성 고려시)
    search_kwargs={"k": 5}          #  상위 5개 관련 사례 검색
)

print(" Retriever 설정 완료")


===> 5. 검색 → 질문과 유사한 문서 찾기
 Retriever 설정 완료


In [20]:

print("===> 6. 생성 → LLM으로 답변 생성")
pythonllm = ChatOpenAI(
    model="gpt-4o",                 #  또는 "gpt-4o-mini" (경제적)
    temperature=0.2,                #  법률 조언은 정확성 중시
    max_tokens=2000                 #  충분한 답변 길이
)


===> 6. 생성 → LLM으로 답변 생성


In [22]:
print("===> 7. 법률 자문 프롬프트 작성")
pythonprompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""


prompt = PromptTemplate(
    template=pythonprompt_template,
    input_variables=["context", "question"]
)
print(" 프롬프트 설정 완료")


===> 7. 법률 자문 프롬프트 작성
 프롬프트 설정 완료


In [ ]:

# ===================================
# 8. QA 체인 생성
# ===================================
print("\n ===> 8.  QA 체인 생성...")

pythonqa_chain = RetrievalQA.from_chain_type(
    llm=pythonllm,                        #  위에서 설정한 LLM
    chain_type="stuff",             #  문서들을 하나로 합쳐서 처리
    retriever=pythonretriever,            #  위에서 설정한 검색기
    chain_type_kwargs={"prompt": prompt},  #  법률 자문 프롬프트
    return_source_documents=True    #  참조 문서도 함께 반환
)



 ===> 8.  QA 체인 생성...


In [24]:

# ===================================
# 9. 테스트 질문 작성
# ===================================
pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]


print("\n" + "=" * 60)
print(" RAG 시스템 테스트")
print("=" * 60)



 RAG 시스템 테스트


In [ ]:

# ===================================
# 10. 질문 및 답변 실행
# ===================================
for i, question in enumerate(pythontest_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = pythonqa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)


【테스트 1/5】
 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 드리겠습니다.

1. **사례 기반 답변**:
   - 제시된 사례 중, 시스템 오류로 인한 아이템 소실에 대한 사례(사례 1-가-1, 2006 및 2009)를 참고할 수 있습니다. 이 사례들에서는 시스템 오류로 인해 아이템이 소실되었으나, 게임 회사는 계정 명의자와의 문제 또는 시스템 오류가 발생하지 않았다는 이유로 복구를 거부하였습니다.

2. **법적 근거 제시**:
   - 일반적으로 게임 회사의 이용약관에 따라 시스템 오류로 인한 피해 보상 여부가 결정됩니다. 따라서 이용약관을 검토하여 시스템 오류로 인한 복구 조항이 있는지 확인하는 것이 중요합니다.
   - 민법 제250조와 제251조는 도난 또는 유실물에 대한 반환 청구에 관한 규정을 두고 있으나, 이는 주로 물리적 재산에 대한 규정이며, 디지털 아이템의 경우에는 적용이 제한적일 수 있습니다.

3. **판례 참조**:
   - 사례 1-가-1(2009)에서는 시스템 오류로 인한 게임머니 소실이 주장되었으나, 당시 유사한 불만이 접수되지 않았고, 사용 내역이 정상적이었기 때문에 시스템 오류로 보기 어렵다고 판단되었습니다.
   - 이러한 판례를 참고할 때, 시스템 오류가 실제로 발생했음을 입증하는 것이 중요합니다.

4. **실무 가이드**:
   - **1단계**: 게임 회사의 고객센터에 공식적으로 문의하여 시스템 오류로 인한 아이템 소실에 대한 복구 요청을 합니다. 이때, 관련 증거(예: 오류 발생 시점의 스크린샷, 로그 기록 등)를 함께 제출합니다.
   - **2단계**: 게임 회사의 이용약관을 검토하여 시스템 오류에

In [ ]:
# ===================================
# 11. 분쟁 유형 분류 함수
# ===================================

def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"
